In [1]:
import torch
from torcheck.stl_prova import (
    Atom, Not, Or, And, Reach, Escape, Somewhere, Everywhere, Surround 
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# 1. Graph initialization
graph_nodes = torch.tensor(list(range(16)))
N = len(graph_nodes)
distance_matrix = torch.zeros((N, N))

edges = [
    (7, 0), (7, 3), (7, 5), (7, 6), (6, 7), (6, 4), (4, 6), (4, 5),
    (6, 1), (6, 9), (9, 6), (9, 2), (9, 13), (9, 8), (8, 9), (8, 13),
    (9, 15), (15, 9), (9, 10), (10, 9), (15, 10), (10, 15), (15, 12),
    (10, 14), (10, 11),
    (0, 7), (3, 7), (5, 7), (5, 4),
    (1, 6), (2, 9), (13, 9), (13, 8),
    (12, 15), (14, 10), (11, 10)
]
for i, j in edges:
    distance_matrix[i, j] = 1.0

# 2. Node features
type_router, type_coord, type_end_dev = 1, 0, 2
node_features = torch.zeros((N, 2))
router_nodes = [4, 6, 7, 8, 10, 15]
coord_node = 9
end_dev_nodes = [0, 1, 2, 3, 5, 11, 12, 13, 14]

for i in router_nodes:
    node_features[i, 0] = type_router
node_features[coord_node, 0] = type_coord
for i in end_dev_nodes:
    node_features[i, 0] = type_end_dev
node_features[:, 1] = torch.rand(N)

# 3. Signal setup
signal = node_features.t().unsqueeze(0).unsqueeze(-1)  # [1, vars, time]

# 4. STL formula
phi1 = Atom(var_index=0, threshold=2.0, lte=None)   # is_end_device
phi2 = Atom(var_index=0, threshold=1.0, lte=None)   # is_router

reach_formula = Reach(
    left_child=phi1,
    right_child=phi2,
    distance_matrix=distance_matrix,
    d1=0,
    d2=1,
    graph_nodes=graph_nodes,
    is_unbounded=False,
    distance_domain_min=0,
    distance_domain_max=1
)

# 5. Evaluation
result = reach_formula.boolean(signal, evaluate_at_all_times=True)

# 6. Output
print("\nVerification: Can each end device reach a router within 1 hop?\n")
for i in end_dev_nodes:
    reachable = result[0, i, 0].item() > 0.5
    print(f"End device {i+1} → {'✅ Yes' if reachable else '❌ No'}")

# 7. Debug matrix
Q = torch.stack([
    torch.arange(N),
    result[0, :, 0],
    (result[0, :, 0] > 0.5).float()
])
print("\nMatrix Q (node, satisfaction, binary):")
print(Q)


Verification: Can each end device reach a router within 1 hop?

End device 1 → ✅ Yes
End device 2 → ✅ Yes
End device 3 → ❌ No
End device 4 → ✅ Yes
End device 6 → ✅ Yes
End device 12 → ✅ Yes
End device 13 → ✅ Yes
End device 14 → ✅ Yes
End device 15 → ✅ Yes

Matrix Q (node, satisfaction, binary):
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15.],
        [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,
          1.,  1.],
        [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,
          1.,  1.]], grad_fn=<StackBackward0>)


In [3]:
# 8. Escape test setup
not_end_device = Not(phi1)  # ¬end_dev

escape_formula = Escape(
    child=not_end_device,
    distance_matrix=distance_matrix,
    d1=3,
    d2=float("inf"),
    graph_nodes=graph_nodes,
    distance_domain_min=0,
    distance_domain_max=float("inf")
)

# 9. Evaluate Escape
escape_result = escape_formula.boolean(signal, evaluate_at_all_times=True)

# 10. Output Escape Results
print("\nVerification: Can each node escape to a non-end-device node at distance ≥ 3?\n")
for i in range(N):
    escapable = escape_result[0, i, 0].item() > 0.5
    print(f"Node {i} → {'✅ Yes' if escapable else '❌ No'}")

# 11. Escape Debug Matrix
escape_Q = torch.stack([
    torch.arange(N),
    escape_result[0, :, 0],
    (escape_result[0, :, 0] > 0.5).float()
])
print("\nEscape Matrix Q (node, satisfaction, binary):")
print(escape_Q)


Verification: Can each node escape to a non-end-device node at distance ≥ 3?

Node 0 → ❌ No
Node 1 → ❌ No
Node 2 → ❌ No
Node 3 → ❌ No
Node 4 → ✅ Yes
Node 5 → ❌ No
Node 6 → ❌ No
Node 7 → ✅ Yes
Node 8 → ✅ Yes
Node 9 → ❌ No
Node 10 → ✅ Yes
Node 11 → ❌ No
Node 12 → ❌ No
Node 13 → ❌ No
Node 14 → ❌ No
Node 15 → ✅ Yes

Escape Matrix Q (node, satisfaction, binary):
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,
          0.,  1.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,
          0.,  1.]], grad_fn=<StackBackward0>)


In [4]:
# 12. Somewhere test setup
coord = Atom(var_index=0, threshold=0.5, lte=True)  # is_coord (type_coord = 0)

somewhere_formula = Somewhere(
    child=coord,
    distance_matrix=distance_matrix,
    d2=2,
    graph_nodes=graph_nodes,
    distance_domain_min=0,
    distance_domain_max=float("inf")
)

# 13. Evaluate Somewhere
somewhere_result = somewhere_formula.boolean(signal, evaluate_at_all_times=True)

# 14. Output Somewhere Results
print("\nVerification: Is there a coordinator within 3 hops from each node?\n")
for i in range(N):
    has_coord_nearby = somewhere_result[0, i, 0].item() > 0.5
    print(f"Node {i} → {'✅ Yes' if has_coord_nearby else '❌ No'}")

# 15. Somewhere Debug Matrix
somewhere_Q = torch.stack([
    torch.arange(N),
    somewhere_result[0, :, 0],
    (somewhere_result[0, :, 0] > 0.5).float()
])
print("\nSomewhere Matrix Q (node, satisfaction, binary):")
print(somewhere_Q)


Verification: Is there a coordinator within 3 hops from each node?

Node 0 → ❌ No
Node 1 → ✅ Yes
Node 2 → ✅ Yes
Node 3 → ❌ No
Node 4 → ✅ Yes
Node 5 → ❌ No
Node 6 → ✅ Yes
Node 7 → ✅ Yes
Node 8 → ✅ Yes
Node 9 → ✅ Yes
Node 10 → ✅ Yes
Node 11 → ✅ Yes
Node 12 → ✅ Yes
Node 13 → ✅ Yes
Node 14 → ✅ Yes
Node 15 → ✅ Yes

Somewhere Matrix Q (node, satisfaction, binary):
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15.],
        [ 0.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.],
        [ 0.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.]], grad_fn=<StackBackward0>)


In [5]:
# 16. Everywhere test setup
router = Atom(var_index=0, threshold=1.0, lte=True)  # is_router (type_router = 1)

everywhere_formula = Everywhere(
    child=router,
    distance_matrix=distance_matrix,
    d2=0,
    graph_nodes=graph_nodes,
    distance_domain_min=0,
    distance_domain_max=float("inf")
)

# 17. Evaluate Everywhere
everywhere_result = everywhere_formula.boolean(signal, evaluate_at_all_times=True)

# 18. Output Everywhere Results
print("\nVerification: Are all nodes within 2 hops routers?\n")
for i in range(N):
    all_routers_nearby = everywhere_result[0, i, 0].item() > 0.5
    print(f"Node {i} → {'✅ Yes' if all_routers_nearby else '❌ No'}")

# 19. Everywhere Debug Matrix
everywhere_Q = torch.stack([
    torch.arange(N),
    everywhere_result[0, :, 0],
    (everywhere_result[0, :, 0] > 0.5).float()
])
print("\nEverywhere Matrix Q (node, satisfaction, binary):")
print(everywhere_Q)


Verification: Are all nodes within 2 hops routers?

Node 0 → ❌ No
Node 1 → ❌ No
Node 2 → ❌ No
Node 3 → ❌ No
Node 4 → ✅ Yes
Node 5 → ❌ No
Node 6 → ✅ Yes
Node 7 → ✅ Yes
Node 8 → ✅ Yes
Node 9 → ✅ Yes
Node 10 → ✅ Yes
Node 11 → ❌ No
Node 12 → ❌ No
Node 13 → ❌ No
Node 14 → ❌ No
Node 15 → ✅ Yes

Everywhere Matrix Q (node, satisfaction, binary):
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
          0.,  1.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
          0.,  1.]], grad_fn=<StackBackward0>)


In [6]:
# 20. Surround test setup
coord_or_router = Or(coord, router)  # is_coord ∨ is_router
end_dev = Atom(var_index=0, threshold=2.0, lte=True)  # is_end_device (type_end_dev = 2)

surround_formula = Surround(
    left_child=coord_or_router,
    right_child=end_dev,
    distance_matrix=distance_matrix,
    d2=4,
    graph_nodes=graph_nodes,
    distance_domain_min=0,
    distance_domain_max=float("inf")
)

# 21. Evaluate Surround
surround_result = surround_formula.boolean(signal, evaluate_at_all_times=True)

# 22. Output Surround Results
print("\nVerification: Is each (coordinator or router) surrounded by end devices within 3 hops?\n")
for i in range(N):
    is_surrounded = surround_result[0, i, 0].item() > 0.5
    print(f"Node {i} → {'✅ Yes' if is_surrounded else '❌ No'}")

# 23. Surround Debug Matrix
surround_Q = torch.stack([
    torch.arange(N),
    surround_result[0, :, 0],
    (surround_result[0, :, 0] > 0.5).float()
])
print("\nSurround Matrix Q (node, satisfaction, binary):")
print(surround_Q)


Verification: Is each (coordinator or router) surrounded by end devices within 3 hops?

Node 0 → ❌ No
Node 1 → ❌ No
Node 2 → ❌ No
Node 3 → ❌ No
Node 4 → ✅ Yes
Node 5 → ❌ No
Node 6 → ✅ Yes
Node 7 → ✅ Yes
Node 8 → ✅ Yes
Node 9 → ✅ Yes
Node 10 → ✅ Yes
Node 11 → ❌ No
Node 12 → ❌ No
Node 13 → ❌ No
Node 14 → ❌ No
Node 15 → ✅ Yes

Surround Matrix Q (node, satisfaction, binary):
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
          0.,  1.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
          0.,  1.]], grad_fn=<StackBackward0>)
